# Setup environment and make sure it works as expected.

This workbook is to validate the environment.

I have built a Docker container based on jupyter/datascience-notebook, with _DifferentialEquations_ and _Plots_ added to the container that I am using locally for these notebooks. Eventually all this should get into this repo, but I am still figureing it out.


In [ ]:
using Pkg
Pkg.status()

Only do the following if you really need to, it can take a long time.

In [ ]:
# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")
# Pkg.add("DiffEqBase")
# Pkg.add("DiffEqOperators")
# Pkg.add("ModelingToolkit")
# Pkg.status()

In [ ]:
using DifferentialEquations
using Plots
using DiffEqBase, DiffEqOperators, LinearAlgebra #, Random
using ModelingToolkit
# using Markdown


## First a sample to make sure that basic package is working as expected.

This is pulled from the _DifferentialEquations_ Docs. (https://diffeq.sciml.ai/stable/)


In [ ]:

f(u,p,t) = 1.01*u
u0 = 1/2
tspan = (0.0,1.0)
prob = ODEProblem(f,u0,tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

plot(sol,linewidth=5,title="Solution to the linear ODE with a thick line",
     xaxis="Time (t)",yaxis="u(t) (in μm)",label="My Thick Line!") # legend=false
plot!(sol.t, t->0.5*exp(1.01t),lw=3,ls=:dash,label="True Solution!")


## And now an example from the book to see if _I_ understand how to use the package.

Solve: $0.3u'' -(1-u^2)u' + u = 0$

In [ ]:

vdp(du, u, p, t) = ((1-u^2)*du - u) / 0.3
vdpspan = (0.0, 20.0)
vdpprob = SecondOrderODEProblem(vdp, 0.0, 1.0, vdpspan)
vdpsol = solve(vdpprob, reltol=1e-8, abstol=1e-8)

plot(vdpsol, vars=(0, 2))


And get the minimas. First Load my utility routines.

In [ ]:
include("localminmax.jl")

**Note:** I am still trying to see how to increase the accuracy here - the book asks for 6 significant digits, but lowering the tolerance in the solver beyond $10^{-8}$ seems to lower the accuracy. This shouldn't be hard (at least, from the book, it seems trivial in MatLab). That said, it appears that I am only seeing an accuracy of about 4 significant digits, which seems wrong given the tolerance specified.

In [ ]:
sigdigits = 6

result = [v[2] for v in vdpsol.u]
mins = localmins(result)
last_min = nothing
for cur_min in mins
    print("t = ", round(vdpsol.t[cur_min[1]], sigdigits=sigdigits),
          " u = ", round(cur_min[2], sigdigits=sigdigits))
    if last_min != nothing
        println("   Diff to last = ", round(vdpsol.t[cur_min[1]] - last_min, sigdigits=sigdigits))
    else
        println("")
    end
    last_min = vdpsol.t[cur_min[1]]
end
println("")

maxs = localmaxs(result)
last_max = nothing
for cur_max in maxs
    print("t = ", round(vdpsol.t[cur_max[1]], sigdigits=sigdigits),
          " u = ", round(cur_max[2], sigdigits=sigdigits))
    if last_max != nothing
        println("   Diff to last = ", round(vdpsol.t[cur_max[1]] - last_max, sigdigits=sigdigits))
    else
        println("")
    end
    last_max = vdpsol.t[cur_max[1]]
end
println("")


Unfortunately we are not getting the same numbers as in the book, they are the same to 3 significant digits, but the book has 5 and the period is certainly not the same to 5 significant digits.

# And try it according to the examples repo.

This code is from the examples Git Repo refered to in the _DifferentialEquations.jl_ docs (look for the Van der Pol examples):
https://github.com/SciML/DiffEqProblemLibrary.jl/blob/master/src/ode/ode_simple_nonlinear_prob.jl


In [ ]:
@parameters t μ
@variables x(t) y(t)
D = Differential(t)
eqs = [D(y) ~ μ*((1-x^2)*y - x),
       D(x) ~ y]
de = ODESystem(eqs; name=:van_der_pol)
van = ODEFunction(de, [y, x], [μ], jac=true)
vspan = (0.0, 20.0)
prob_ode_vanderpol = ODEProblem(van, [0.0; 1.0], vspan, [1/0.3])
vsol = solve(prob_ode_vanderpol, reltol=1e-8, abstol=1e-8)
plot(vsol, vars=(0, 2))

In [ ]:
sigdigits = 6

result = [v[2] for v in vsol.u]
mins = localmins(result)
last_min = nothing
for cur_min in mins
    print("t = ", round(vsol.t[cur_min[1]], sigdigits=sigdigits),
          " u = ", round(cur_min[2], sigdigits=sigdigits))
    if last_min != nothing
        println("   Diff to last = ", round(vsol.t[cur_min[1]] - last_min, sigdigits=sigdigits))
    else
        println("")
    end
    last_min = vsol.t[cur_min[1]]
end
println("")

maxs = localmaxs(result)
last_max = nothing
for cur_max in maxs
    print("t = ", round(vsol.t[cur_max[1]], sigdigits=sigdigits),
          " u = ", round(cur_max[2], sigdigits=sigdigits))
    if last_max != nothing
        println("   Diff to last = ", round(vsol.t[cur_max[1]] - last_max, sigdigits=sigdigits))
    else
        println("")
    end
    last_max = vsol.t[cur_max[1]]
end
println("")


Well, that looks like we actually got it aligning with the book. And I think I like the syntax better.
